NAMES & IDs: 
Bryant Liu - 
Jingyao Chen - A92071265
Joseph D’Angelo - A13580892
Tammy Lee - A13573330

OVERVIEW: 
Include 3-4 sentences summarizing your group’s project 

## RESEARCH QUESTION: 
~~What is your research question? (1-2 sentences)~~  
Do wealth and geographical location have an impact on the types of financial complains and the quantity of complaints submitted?

## HYPOTHESIS:   
~~What is your main hypothesis and predictions? Briefly explain why. (2-3 sentences)~~  
We believe that lower income areas will submit complaints that differ in type and quantity from higher income areas. We also believe there will be a correlation between the type of area you live in (city, suburb, country) and the type of complaints submitted. Our reason is that people living in different income areas might pay for different types of services and therefore might encounter different types of issues when using their financial products. 

## BACKGROUND & PRIOR WORK: 
Why is this question of interest to your group? What background information led you to your hypothesis. Why is this important?  
Find some relevant prior work, and reference those sources. Even if you think you have a totally novel question, find the most similar prior work that you can and discuss how it relates to your project. 
References can be research publications, but they need not be. Blogs, GitHub repositories, company websites, etc., are all viable references if they are relevant to your project. (2-3 paragraphs, including at least 2 references) 




## DATASET(S): 
~~What data will you use to answer your question? Describe the dataset(s) in terms of number of observations, what kind of features it contains, etc. You must use at least one dataset containing at least approximately 1000 observations (if your data are smaller but you feel they are sufficient, email Prof. Ellis). You are welcome (and in fact recommended) to find multiple datasets! If you do so, describe each one, and briefly explain how you will combine them together. Include the source of the dataset in the description here.~~ 

The ZIP code data show all kinds of information about income and tax items classified by state, ZIP code, and size of adjusted gross income. We used the 2016 California ZIP code database which has 1598 zip codes.
https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi

We used the Consumer Complaint database in data.gov. It displays the consumer complaints about financial products and services received by the Consumer Financial Protection Bureau (CFPB), a U.S. government agency that protects consumers from unfair, deceptive, or abusive practices and take action against companies that break the law. The database contains over 100,000 complaints from years 2017-2019 and is refreshed daily. 
https://catalog.data.gov/dataset/consumer-complaint-database#topic=consumer_navigation

## DATA CLEANING: 
What methods did you use to analyze your data? Briefly explain what steps you had to take before you were able to use the datasets you chose to answer your question of interest. 
- How 'clean' is the data? - What did you have to do to get the data into a usable format? - What pre-processing steps that were required for your methods (for example, checking data distributions 
and performing any transformations that may be required) 



Consumer Complaints Database cleaning:
We did pre process cleaning by choosing the data set that had the most recent and full database of consumer complaints. The data was somewhat clean when we got it, the data set contained a good amount of blank rows as therefore we needed to clean those lines out. 
For the columns of interest, we had some distribution issues within some of those columns. There are data that has too few occurrences in terms of complaints category, which we then removed as it is not representative enough.
For the overall database, we cleaned it by removing data we didn’t want as well as data rows that are incomplete. First we dropped all rows not from the state of California, this is done not only to remove data that we are not considering, but also to reduce the dataset’s csv file size from over .4GB to under 15MB. Dropping all non-California states is done by using the given column for states within the dataframe. Then we dropped all the columns that we are not referencing from, this is done first because we do not want to drop rows that had blank data but in columns that we are not interested in. 
For individual columns cleaning, we targeted each column of interest one by one. The first column was the zipcode, we cleaned this by first dropping all rows with blank zip codes. After that we realized there are some zip code data with X’s in them, which meant that the person that submitted the complaint chose the option to hide their zip code. We cleaned out this problem by dropping all lines with checking if ‘X’ exist in the zip code. 
After all the data cleaning so far, we saved the resulting data cleaning file in a new csv file to import into jupyter-notebooks later on. This is suggested to do on a local computer as the original dataset is a very large file, GitHub will not accept such a large file size originally. 


There are also data points that either doesn't make sense or too extreme of outliers, which we cleaned off. Such 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
df_income = pd.read_csv('irs-zipcode.csv')

# Change column name using second row
df_income.columns = df_income.iloc[2]

# Drop 0th, 1st, nan rows; select 'Number of returns' and 'Amount' of 'Total income' columns 
df_income = df_income.drop([0, 1])
df_income = df_income.rename(index=str, columns={'ZIP\ncode [1]': 'ZIP code'})
df_income = df_income.rename(index=str, columns={'Number of returns': 'Number'})
df_income = df_income.set_index('ZIP code')

#df_income = df_income.drop(['ZIP\ncode [1]'])
df_income_selected = df_income.iloc[:, [16, 17]]
df_income_selected = df_income_selected.dropna()
df_income_selected.columns = ['Number of returns', 'Amount']

# Drop zip code 00000 and 99999
df_income_selected = df_income_selected.drop(['00000', '99999'])

# Drop all rows but the first, which is total returns and amount
df_income_selected = df_income_selected.loc[~df_income_selected.index.duplicated(keep='first')]
df_income_selected = df_income_selected.reset_index()
df_income_selected = df_income_selected.drop([0])
df_income_selected = df_income_selected.set_index('ZIP code')
df_income_selected = df_income_selected.rename(index=str, columns={'Number of returns': 'Number'})
df_income_selected['Amount'] = df_income_selected['Amount'].str.replace(',', '')
df_income_selected['Number'] = df_income_selected['Number'].str.replace(',', '')
df_income_selected['Amount'] = pd.to_numeric(df_income_selected['Amount'])
df_income_selected['Number'] = pd.to_numeric(df_income_selected['Number'])


df_income_selected['Wealth'] = df_income_selected.apply(lambda x: (x.Amount / x.Number) * 1000, axis=1)
df_income = df_income_selected
df_income

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Number,Amount,Wealth
ZIP code,,,
90001,21670,619635,28594.139363
90002,19890,563172,28314.328808
90003,27290,730810,26779.406376
90004,27810,1995586,71757.856886
90005,15850,807743,50961.703470
90006,23010,685595,29795.523685
90007,12620,382443,30304.516640
90008,15040,791390,52619.015957
90010,2430,533018,219348.971193


In [3]:
df_complaints = pd.read_csv('new2.csv')

# Remove all of the unimportant columns from our dataframe
df_complaints = df_complaints.drop(columns=['Date received', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative',
                                           'Company public response', 'Company', 'State', 'Tags', 'Consumer consent provided?',
                                           'Submitted via', 'Date sent to company', 'Company response to consumer', 
                                           'Timely response?', 'Consumer disputed?', 'Complaint ID'])

# Remove weird zip codes
df_complaints = df_complaints[df_complaints['ZIP code'] >= 90001]
df_complaints = df_complaints.sort_index()

# Create groups for unique zip codes and unique products
df_complaints_groups = df_complaints.groupby(['ZIP code', 'Product'])
df_complaints_2 = pd.DataFrame(columns = ['ZIP code'])

# In each group, get the length which represents 
for key, item in df_complaints_groups:

    group_length = len(df_complaints_groups.get_group(key))
    df_complaints_2 = df_complaints_2.append({'ZIP code' : int(key[0]), key[1] : group_length}, ignore_index=True)

df_complaints_2 = df_complaints_2.fillna(int(0))

# Sum all unique zip codes to get one row per zip code
grouped_df = df_complaints_2.groupby('ZIP code').sum()

# Print the dataframe
grouped_df


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Checking or savings account,Consumer Loan,Credit reporting,"Credit reporting, credit repair services, or other personal consumer reports",Debt collection,Mortgage,Credit card or prepaid card,Student loan,Bank account or service,"Payday loan, title loan, or personal loan",Vehicle loan or lease,Credit card,"Money transfer, virtual currency, or money service",Payday loan,Prepaid card,Money transfers,Other financial service
ZIP code,,,,,,,,,,,,,,,,,
90001.0,3.0,1.0,4.0,28.0,22.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90002.0,0.0,0.0,0.0,21.0,13.0,3.0,4.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90003.0,3.0,0.0,0.0,40.0,9.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90004.0,7.0,0.0,0.0,13.0,8.0,3.0,5.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
90005.0,1.0,0.0,5.0,5.0,7.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
90006.0,5.0,0.0,0.0,10.0,2.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
90007.0,4.0,0.0,0.0,14.0,12.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
90008.0,7.0,0.0,0.0,48.0,13.0,10.0,1.0,3.0,2.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
90009.0,4.0,0.0,0.0,24.0,6.0,3.0,5.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0


In [44]:
grouped_df.index = grouped_df.index.astype('str')
df_income.index = df_income.index.astype('str')

complaints_df = pd.concat([grouped_df, df_income], axis=1, sort=False)
complaints_df = complaints_df.dropna()
complaints_df['Total Complaints'] = grouped_df.sum(axis=1)

complaints_df

,Checking or savings account,Consumer Loan,Credit reporting,"Credit reporting, credit repair services, or other personal consumer reports",Debt collection,Mortgage,Credit card or prepaid card,Student loan,Bank account or service,"Payday loan, title loan, or personal loan",...,Credit card,"Money transfer, virtual currency, or money service",Payday loan,Prepaid card,Money transfers,Other financial service,Number,Amount,Wealth,Total Complaints
90001,3.0,1.0,4.0,28.0,22.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,21670.0,619635.0,28594.139363,64
90002,0.0,0.0,0.0,21.0,13.0,3.0,4.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19890.0,563172.0,28314.328808,48
90003,3.0,0.0,0.0,40.0,9.0,4.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27290.0,730810.0,26779.406376,59
90004,7.0,0.0,0.0,13.0,8.0,3.0,5.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27810.0,1995586.0,71757.856886,41
90005,1.0,0.0,5.0,5.0,7.0,3.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,15850.0,807743.0,50961.703470,23
90006,5.0,0.0,0.0,10.0,2.0,3.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,23010.0,685595.0,29795.523685,23
90007,4.0,0.0,0.0,14.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,12620.0,382443.0,30304.516640,33
90008,7.0,0.0,0.0,48.0,13.0,10.0,1.0,3.0,2.0,1.0,...,1.0,2.0,0.0,0.0,0.0,0.0,15040.0,791390.0,52619.015957,88
90010,1.0,0.0,0.0,5.0,3.0,0.0,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2430.0,533018.0,219348.971193,12
90011,1.0,0.0,1.0,6.0,16.0,7.0,1.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,38350.0,1026864.0,26776.114733,35


DATA ANALYSIS & RESULTS: This section should include markdown text and code walking us through the following: 
- EDA - What distributions do your variables take? 
- Are there any outliers? - Relationship between variables? - Analysis (Note that you will likely have to do some Googling for analytical approaches not discussed in 
class. This is expected for this project and an important skill for a data scientist to master.) 
- What approaches did you use? Why? - What were the results? - What were your interpretation of these findings. 


There must be at least three appropriate data visualizations​throughout these sections. Each visualization must included an interpretation of what is displayed *and* what should be learned from that visualization. Be sure that the appropriate type of visualization is generated given the data that you have, axes are all labeled, and the visualizations clearly communicate the point you’re trying to make. 

ETHICS & PRIVACY: 
Briefly acknowledge and address any potential issues of ethics and privacy for the proposed project. In particular: 
- Did you have permission to use this data, for this purpose? - Are there privacy concerns regarding your datasets that you need to deal with, and/or terms of use that 
you need to comply with? - Are there potential biases in your dataset(s), in terms of who it composes, and how it was collected, that 
may be problematic in terms of it allowing for equitable analysis? (For example, does your data exclude particular populations, or is it likely to reflect particular human biases in a way that could be a problem?) - Are there any other issues related to your topic area, data, and/or analyses that are potentially problematic 
in terms of data privacy and equitable impact? - How did you handle issues you identified? 
(1-2 paragraphs) 


CONCLUSION & DISCUSSION: Discuss your project. Summarize your data and question. Briefly describe your analysis. Summarize your results and conclusions. Be sure to mention any limitations of your project. Discuss the impact of this work on society. (2-3 paragraphs) 